In [15]:
import sqlite3
from sqlite3 import Error
import requests

In [4]:
inventory_sheet_url = 'https://docs.google.com/spreadsheets/d/1abfWjKouFxD_erzhdnam8tCtFC5G_956/edit#gid=2096778584'

In [3]:
def get_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return conn

In [9]:
def get_sheet_data(sheet_url):
    """Returns 2d array of sheet data"""
    sheet_id = sheet_url.split('/')[5]
    gid = sheet_url.split('/')[6].split('=')[1]
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=tsv&gid={gid}"
    response = requests.get(url)
    data = response.text
    rows = data.split('\n')
    sheet_data = []
    for row in rows:
        sheet_data.append(row.split('\t'))
    return sheet_data

In [17]:
def parseDate(date):
    """MM/DD/YYYY to YYYY-MM-DD"""
    m, d, y = date.split('/')
    return f"{y}-{m}-{d}"

In [10]:
inv = get_sheet_data(inventory_sheet_url)


In [16]:
conn = get_connection('db.sqlite3')
cur = conn.cursor()

Connection to SQLite DB successful


In [18]:
for row in inv[1:]:
    inventoryID = row[0]
    scheduleID = row[1]
    departureDate = parseDate(row[4])
    arrivalDate = parseDate(row[5])
    totalCapacity = row[8]
    totalInventory = row[9]
    bookedInventory = row[10]
    cur.execute(f"""
        INSERT INTO Inventory VALUES (
            '{inventoryID}',
            '{scheduleID}',
            '{departureDate}',
            '{arrivalDate}',
            {totalCapacity},
            {totalInventory},
            {bookedInventory}
        );
    """)
    conn.commit()
